# Module 3：卷積神經網路 (CNN) 與影像任務

## 學習目標

1. 理解卷積運算的原理與直覺
2. 掌握 CNN 的核心組件：卷積層、池化層
3. 了解為什麼 CNN 適合處理影像
4. 學會資料增強技術
5. 理解並實作 ResNet 的殘差連接
6. 實作：用 CNN 分類 CIFAR-10

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

plt.rcParams['figure.figsize'] = (12, 4)

---

## Part 1：為什麼需要 CNN？

### 1.1 MLP 處理影像的問題

假設用 MLP 處理 224×224 的 RGB 影像：
- 輸入維度：224 × 224 × 3 = **150,528**
- 如果第一層有 1000 個神經元：150,528 × 1000 = **1.5 億參數**！

**問題：**
1. 參數太多，容易過擬合
2. 沒有利用影像的空間結構
3. 不具備平移不變性（同一物體在不同位置要重新學）

In [ ]:
# 參數量比較

# MLP 處理 32x32 RGB 影像
input_size = 32 * 32 * 3  # 3072
hidden_size = 512

mlp_params = input_size * hidden_size + hidden_size  # 第一層
print(f"MLP 第一層參數量: {mlp_params:,}")

# CNN 處理同樣影像
# 3x3 kernel, 3 input channels, 64 output channels
cnn_params = 3 * 3 * 3 * 64 + 64  # kernel weights + bias
print(f"CNN 第一層參數量: {cnn_params:,}")

print(f"\nCNN 參數量只有 MLP 的 {cnn_params/mlp_params*100:.2f}%！")

### 1.2 CNN 的核心思想

1. **局部連接 (Local Connectivity)**：每個神經元只看輸入的一小塊區域
2. **權重共享 (Weight Sharing)**：同一個 filter 在整張圖上滑動
3. **平移不變性 (Translation Invariance)**：同一物體在不同位置會被同樣識別

---

## Part 2：卷積運算 (Convolution)

### 2.1 卷積的直覺

**概念：** 用一個小的 filter（濾波器/kernel）在圖片上「滑動」，每個位置做內積。

```
輸入圖片          Filter          輸出
┌─┬─┬─┬─┐       ┌─┬─┬─┐
│1│2│3│0│       │1│0│1│      滑動並做內積
├─┼─┼─┼─┤   ×   ├─┼─┼─┤   →   得到 feature map
│0│1│2│3│       │0│1│0│
├─┼─┼─┼─┤       ├─┼─┼─┤
│3│0│1│2│       │1│0│1│
├─┼─┼─┼─┤       └─┴─┴─┘
│2│3│0│1│
└─┴─┴─┴─┘
```

In [ ]:
# 卷積運算視覺化

def visualize_convolution(image, kernel, title="Convolution"):
    """視覺化卷積運算"""
    # 轉成 tensor 並加上 batch 和 channel 維度
    img_tensor = torch.tensor(image, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    kernel_tensor = torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    # 執行卷積
    output = F.conv2d(img_tensor, kernel_tensor, padding=0)
    
    # 視覺化
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Input Image')
    
    axes[1].imshow(kernel, cmap='gray')
    axes[1].set_title('Kernel/Filter')
    
    axes[2].imshow(output.squeeze().numpy(), cmap='gray')
    axes[2].set_title('Output (Feature Map)')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    
    return output.squeeze().numpy()

# 建立簡單的測試圖片（有垂直邊緣）
image = np.array([
    [0, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1],
    [0, 0, 0, 1, 1, 1],
    ], dtype=np.float32)

# 垂直邊緣檢測 kernel
vertical_edge_kernel = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1],
], dtype=np.float32)

output = visualize_convolution(image, vertical_edge_kernel, "Vertical Edge Detection")
print("\n觀察：垂直邊緣（從暗到亮）產生正值，被檢測出來了！")

In [ ]:
# 不同的 kernel 效果

# 載入真實圖片
from PIL import Image
import urllib.request

# 使用 torchvision 的範例圖片或建立一個簡單的測試圖片
# 建立一個有邊緣的合成圖片
np.random.seed(42)
test_image = np.zeros((64, 64), dtype=np.float32)
test_image[16:48, 16:48] = 1.0  # 中間一個白色方塊
test_image += np.random.randn(64, 64) * 0.1  # 加點噪音
test_image = np.clip(test_image, 0, 1)

# 定義不同的 kernel
kernels = {
    'Identity': np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]]),
    'Edge Detection': np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]),
    'Sharpen': np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
    'Blur': np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9.0,
    'Sobel X': np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]),
    'Sobel Y': np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]),
}

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

# 原圖
axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

# 各種 kernel 效果
for i, (name, kernel) in enumerate(kernels.items()):
    img_tensor = torch.tensor(test_image).unsqueeze(0).unsqueeze(0)
    kernel_tensor = torch.tensor(kernel, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    output = F.conv2d(img_tensor, kernel_tensor, padding=1)
    
    axes[i+1].imshow(output.squeeze().numpy(), cmap='gray')
    axes[i+1].set_title(name)
    axes[i+1].axis('off')

axes[-1].axis('off')  # 隱藏多餘的 subplot
plt.suptitle('Different Convolution Kernels', fontsize=14)
plt.tight_layout()
plt.show()

### 2.2 卷積的重要參數

| 參數 | 說明 | 常見值 |
|------|------|--------|
| **kernel_size** | filter 的大小 | 3×3, 5×5, 7×7 |
| **stride** | 每次滑動的步長 | 1, 2 |
| **padding** | 在邊緣補零 | 0, 1, 'same' |
| **in_channels** | 輸入通道數 | RGB=3, 灰階=1 |
| **out_channels** | 輸出通道數（filter 數量） | 32, 64, 128... |

In [ ]:
# 卷積參數示範

# 輸入：batch=1, channels=3 (RGB), height=32, width=32
x = torch.randn(1, 3, 32, 32)
print(f"輸入 shape: {x.shape}")

# 不同參數的卷積層
conv_configs = [
    {'kernel_size': 3, 'stride': 1, 'padding': 0},  # 基本
    {'kernel_size': 3, 'stride': 1, 'padding': 1},  # same padding
    {'kernel_size': 3, 'stride': 2, 'padding': 1},  # stride=2 降維
    {'kernel_size': 5, 'stride': 1, 'padding': 2},  # 大 kernel
    {'kernel_size': 7, 'stride': 2, 'padding': 3},  # 更大 kernel
]

print("\n不同參數的輸出 shape：")
for config in conv_configs:
    conv = nn.Conv2d(in_channels=3, out_channels=64, **config)
    out = conv(x)
    print(f"  kernel={config['kernel_size']}, stride={config['stride']}, "
          f"padding={config['padding']} -> {out.shape}")

In [ ]:
# 輸出尺寸公式
# output_size = (input_size - kernel_size + 2*padding) / stride + 1

def calc_output_size(input_size, kernel_size, stride, padding):
    return (input_size - kernel_size + 2*padding) // stride + 1

print("輸出尺寸公式: output = (input - kernel + 2*padding) / stride + 1")
print("\n範例 (input=32):")

for config in conv_configs:
    out_size = calc_output_size(32, config['kernel_size'], 
                                config['stride'], config['padding'])
    print(f"  k={config['kernel_size']}, s={config['stride']}, "
          f"p={config['padding']} -> output={out_size}")

### 2.3 多通道卷積

**RGB 圖片的卷積：**
- 輸入：3 個 channel（R, G, B）
- 每個 filter 有 3 個對應的 kernel
- 3 個 kernel 分別和 3 個 channel 做卷積，結果相加
- 一個 filter 產生一個 output channel

In [ ]:
# 多通道卷積視覺化

print("多通道卷積的維度變化：")
print("")
print("輸入: (batch, in_channels, H, W)")
print("       (1,     3,          32, 32)")
print("")
print("Conv2d(in_channels=3, out_channels=64, kernel_size=3)")
print("       ↓")
print("Weight shape: (out_channels, in_channels, kH, kW)")
print("              (64,          3,           3,  3)")
print("       ↓")
print("輸出: (batch, out_channels, H', W')")
print("       (1,     64,          30, 30)")

# 驗證
conv = nn.Conv2d(3, 64, kernel_size=3)
print(f"\n實際 weight shape: {conv.weight.shape}")
print(f"總參數量: {conv.weight.numel() + conv.bias.numel()}")
print(f"  = 64 × 3 × 3 × 3 + 64 = {64*3*3*3 + 64}")

---

## Part 3：池化層 (Pooling)

### 3.1 為什麼需要池化？

1. **降低維度**：減少計算量和參數
2. **增加感受野**：讓每個神經元「看到」更大的範圍
3. **平移不變性**：小範圍的位移不影響結果

### 3.2 Max Pooling vs Average Pooling

In [ ]:
# 池化示範

# 建立範例輸入
x = torch.tensor([[[[1., 2., 3., 4.],
                    [5., 6., 7., 8.],
                    [9., 10., 11., 12.],
                    [4., 3., 2., 1.]]]])

print(f"輸入 (4×4):\n{x.squeeze()}")

# Max Pooling
max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
max_out = max_pool(x)
print(f"\nMax Pooling 2×2 (取最大值):\n{max_out.squeeze()}")

# Average Pooling
avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
avg_out = avg_pool(x)
print(f"\nAverage Pooling 2×2 (取平均):\n{avg_out.squeeze()}")

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(x.squeeze(), cmap='viridis')
axes[0].set_title('Input (4×4)')
for i in range(4):
    for j in range(4):
        axes[0].text(j, i, f'{x[0,0,i,j]:.0f}', ha='center', va='center', color='white')

axes[1].imshow(max_out.squeeze(), cmap='viridis')
axes[1].set_title('Max Pool (2×2)')
for i in range(2):
    for j in range(2):
        axes[1].text(j, i, f'{max_out[0,0,i,j]:.0f}', ha='center', va='center', color='white')

axes[2].imshow(avg_out.squeeze(), cmap='viridis')
axes[2].set_title('Avg Pool (2×2)')
for i in range(2):
    for j in range(2):
        axes[2].text(j, i, f'{avg_out[0,0,i,j]:.1f}', ha='center', va='center', color='white')

plt.tight_layout()
plt.show()

### 3.3 Global Average Pooling

**概念：** 對整個 feature map 取平均，常用在網路最後取代全連接層。

**優點：**
- 大幅減少參數
- 對輸入尺寸更彈性

In [ ]:
# Global Average Pooling

x = torch.randn(1, 512, 7, 7)  # 典型的 CNN 輸出
print(f"輸入 shape: {x.shape}")

# 傳統做法：Flatten + Linear
flatten = nn.Flatten()
fc = nn.Linear(512 * 7 * 7, 10)
out_fc = fc(flatten(x))
print(f"Flatten + FC: {512*7*7} -> 10, 參數量 = {512*7*7*10 + 10:,}")

# Global Average Pooling + Linear
gap = nn.AdaptiveAvgPool2d(1)  # 輸出 1×1
fc_small = nn.Linear(512, 10)
out_gap = fc_small(gap(x).view(1, -1))
print(f"GAP + FC: 512 -> 10, 參數量 = {512*10 + 10:,}")

---

## Part 4：經典 CNN 架構

### 4.1 基本 CNN 結構

```
Input → [Conv → ReLU → Pool] × N → Flatten → FC → Output
```

In [ ]:
# 簡單的 CNN

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 卷積層
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # 池化層
        self.pool = nn.MaxPool2d(2, 2)
        
        # 全連接層
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Conv block 1: 32x32 -> 16x16
        x = self.pool(F.relu(self.conv1(x)))
        
        # Conv block 2: 16x16 -> 8x8
        x = self.pool(F.relu(self.conv2(x)))
        
        # Conv block 3: 8x8 -> 4x4
        x = self.pool(F.relu(self.conv3(x)))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x

# 測試
model = SimpleCNN()
x = torch.randn(1, 3, 32, 32)
print(f"輸入: {x.shape}")
print(f"輸出: {model(x).shape}")

# 參數量
total_params = sum(p.numel() for p in model.parameters())
print(f"\n總參數量: {total_params:,}")

In [ ]:
# 追蹤每一層的輸出 shape

def trace_shapes(model, input_shape=(1, 3, 32, 32)):
    """追蹤模型每一層的輸出 shape"""
    x = torch.randn(input_shape)
    print(f"Input: {x.shape}")
    print("-" * 40)
    
    # Conv blocks
    x = model.pool(F.relu(model.conv1(x)))
    print(f"After conv1 + pool: {x.shape}")
    
    x = model.pool(F.relu(model.conv2(x)))
    print(f"After conv2 + pool: {x.shape}")
    
    x = model.pool(F.relu(model.conv3(x)))
    print(f"After conv3 + pool: {x.shape}")
    
    x = x.view(x.size(0), -1)
    print(f"After flatten: {x.shape}")
    
    x = F.relu(model.fc1(x))
    print(f"After fc1: {x.shape}")
    
    x = model.fc2(x)
    print(f"After fc2 (output): {x.shape}")

trace_shapes(model)

### 4.2 現代 CNN：使用 BatchNorm

**Conv → BatchNorm → ReLU** 是現代 CNN 的標準組合

In [ ]:
# 現代 CNN 設計

class ModernCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Conv block helper
        def conv_block(in_ch, out_ch):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, 3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, 2)
            )
        
        self.features = nn.Sequential(
            conv_block(3, 64),      # 32 -> 16
            conv_block(64, 128),    # 16 -> 8
            conv_block(128, 256),   # 8 -> 4
        )
        
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = ModernCNN()
print(model)
print(f"\n總參數量: {sum(p.numel() for p in model.parameters()):,}")

---

## Part 5：殘差連接 (Residual Connection)

### 5.1 為什麼需要殘差連接？

**問題：** 網路越深，訓練越困難（梯度消失/爆炸、退化問題）

**ResNet 的解決方案：** 讓網路學習「殘差」而不是完整的映射

```
原本要學：H(x)
改成學：F(x) = H(x) - x （殘差）
輸出：H(x) = F(x) + x （加回 x）
```

**直覺：** 如果什麼都不學，至少可以傳遞 x（identity），不會比淺網路差

In [ ]:
# 殘差連接視覺化

print("""
傳統網路：
    x → [Conv → BN → ReLU → Conv → BN] → ReLU → y
    
    y = F(x)

殘差網路 (ResNet)：
    x → [Conv → BN → ReLU → Conv → BN] → (+) → ReLU → y
    └──────────────────────────────────────┘
                    Skip Connection
    
    y = F(x) + x
""")

In [ ]:
# 實作 Residual Block

class ResidualBlock(nn.Module):
    """基本的殘差區塊"""
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        
        # 主要路徑
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection（如果維度不同需要調整）
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, 
                          stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        # 主要路徑
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # 加上 skip connection
        out += self.shortcut(x)
        
        # 最後的 ReLU
        out = F.relu(out)
        
        return out

# 測試
block = ResidualBlock(64, 64)
x = torch.randn(1, 64, 32, 32)
print(f"Same dimensions: {x.shape} -> {block(x).shape}")

block_down = ResidualBlock(64, 128, stride=2)
print(f"Downsampling: {x.shape} -> {block_down(x).shape}")

In [ ]:
# 小型 ResNet

class SmallResNet(nn.Module):
    """簡化版 ResNet for CIFAR-10"""
    def __init__(self, num_classes=10):
        super().__init__()
        
        # 初始卷積
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # 殘差層
        self.layer1 = self._make_layer(64, 64, 2, stride=1)   # 32x32
        self.layer2 = self._make_layer(64, 128, 2, stride=2)  # 16x16
        self.layer3 = self._make_layer(128, 256, 2, stride=2) # 8x8
        
        # 分類器
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)
    
    def _make_layer(self, in_ch, out_ch, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_ch, out_ch, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_ch, out_ch))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

resnet = SmallResNet()
x = torch.randn(1, 3, 32, 32)
print(f"輸入: {x.shape}")
print(f"輸出: {resnet(x).shape}")
print(f"\n總參數量: {sum(p.numel() for p in resnet.parameters()):,}")

---

## Part 6：資料增強 (Data Augmentation)

### 6.1 為什麼需要資料增強？

- 增加訓練資料的多樣性
- 減少過擬合
- 讓模型學到更穩健的特徵

### 6.2 常用的影像增強方法

In [ ]:
# 資料增強示範

# 載入一張範例圖片
transform_basic = transforms.Compose([
    transforms.ToTensor(),
])

# 載入 CIFAR-10 的一張圖
cifar_temp = torchvision.datasets.CIFAR10(root='./data', train=True, 
                                           download=True, transform=transform_basic)
sample_img, label = cifar_temp[0]
sample_pil = transforms.ToPILImage()(sample_img)

# 各種增強方法
augmentations = {
    'Original': transforms.Compose([]),
    'HorizontalFlip': transforms.RandomHorizontalFlip(p=1.0),
    'Rotation': transforms.RandomRotation(30),
    'ColorJitter': transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    'RandomCrop': transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
    'GaussianBlur': transforms.GaussianBlur(3),
}

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, (name, aug) in enumerate(augmentations.items()):
    if name == 'Original':
        img = sample_pil
    else:
        torch.manual_seed(42)  # 固定隨機種子方便展示
        img = aug(sample_pil)
    
    axes[i].imshow(img)
    axes[i].set_title(name)
    axes[i].axis('off')

plt.suptitle('Data Augmentation Examples', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 實際使用的增強組合

# 訓練時的增強（較強）
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),      # 隨機裁剪
    transforms.RandomHorizontalFlip(),          # 隨機水平翻轉
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # 顏色抖動
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # CIFAR-10 的均值
                         (0.2023, 0.1994, 0.2010))   # CIFAR-10 的標準差
])

# 測試時不增強
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# 展示同一張圖的多次增強結果
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    # 每次隨機增強
    aug_img = train_transform(sample_pil)
    # 反正規化以便顯示
    aug_img = aug_img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    aug_img = aug_img + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    aug_img = aug_img.clamp(0, 1)
    
    axes[i].imshow(aug_img.permute(1, 2, 0))
    axes[i].axis('off')

plt.suptitle('Same Image with Random Augmentations', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 7：完整實作 - CIFAR-10 分類

把所有概念結合，訓練一個完整的 CNN 來分類 CIFAR-10。

In [ ]:
# 載入 CIFAR-10

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, 
                          num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False,
                         num_workers=2, pin_memory=True)

# CIFAR-10 類別
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

print(f"訓練集: {len(train_dataset)} 張")
print(f"測試集: {len(test_dataset)} 張")
print(f"類別: {classes}")

In [ ]:
# 顯示一些範例圖片

def imshow(img, title=None):
    """顯示圖片（反正規化）"""
    img = img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    img = img + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    img = img.clamp(0, 1)
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    if title:
        plt.title(title)

# 取一批資料
dataiter = iter(train_loader)
images, labels = next(dataiter)

# 顯示前 16 張
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i, ax in enumerate(axes.flatten()):
    img = images[i]
    img = img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    img = img + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    img = img.clamp(0, 1)
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(classes[labels[i]])
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# 建立模型
model = SmallResNet(num_classes=10).to(device)

# 損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

# 學習率調度器
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 訓練函數

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / total, 100. * correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return total_loss / total, 100. * correct / total

In [ ]:
# 訓練！

num_epochs = 20
history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'lr': []}

print("開始訓練...")
print("=" * 70)

best_acc = 0
for epoch in range(num_epochs):
    current_lr = optimizer.param_groups[0]['lr']
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    history['lr'].append(current_lr)
    
    # 保存最佳模型
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_cifar10_model.pth')
    
    print(f"Epoch [{epoch+1:2d}/{num_epochs}] "
          f"LR: {current_lr:.6f} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
          f"Test Loss: {test_loss:.4f}, Acc: {test_acc:.2f}%")

print("=" * 70)
print(f"最佳測試準確率: {best_acc:.2f}%")

In [ ]:
# 視覺化訓練過程

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
ax = axes[0]
ax.plot(history['train_loss'], label='Train')
ax.plot(history['test_loss'], label='Test')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Loss Curve')
ax.legend()
ax.grid(True)

# Accuracy
ax = axes[1]
ax.plot(history['train_acc'], label='Train')
ax.plot(history['test_acc'], label='Test')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Accuracy Curve')
ax.legend()
ax.grid(True)

# Learning Rate
ax = axes[2]
ax.plot(history['lr'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Learning Rate')
ax.set_title('Learning Rate Schedule (Cosine)')
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 查看預測結果

model.eval()
images, labels = next(iter(test_loader))
images, labels = images.to(device), labels.to(device)

with torch.no_grad():
    outputs = model(images)
    probs = F.softmax(outputs, dim=1)
    _, predicted = outputs.max(1)

# 顯示前 16 個預測
fig, axes = plt.subplots(2, 8, figsize=(16, 5))
for i, ax in enumerate(axes.flatten()):
    img = images[i].cpu()
    img = img * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    img = img + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    img = img.clamp(0, 1)
    
    pred_class = classes[predicted[i]]
    true_class = classes[labels[i]]
    prob = probs[i, predicted[i]].item()
    
    color = 'green' if predicted[i] == labels[i] else 'red'
    
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(f'P: {pred_class}\nT: {true_class}\n{prob:.1%}', 
                 color=color, fontsize=9)
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 每個類別的準確率

class_correct = [0] * 10
class_total = [0] * 10

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        
        for i in range(labels.size(0)):
            label = labels[i].item()
            class_total[label] += 1
            if predicted[i] == labels[i]:
                class_correct[label] += 1

print("每個類別的準確率：")
print("-" * 30)
for i in range(10):
    acc = 100 * class_correct[i] / class_total[i]
    print(f"{classes[i]:>10}: {acc:.1f}%")

# 視覺化
plt.figure(figsize=(10, 5))
accs = [100 * class_correct[i] / class_total[i] for i in range(10)]
bars = plt.bar(classes, accs)
plt.ylabel('Accuracy (%)')
plt.title('Per-Class Accuracy')
plt.xticks(rotation=45)

# 標記最低和最高
min_idx = np.argmin(accs)
max_idx = np.argmax(accs)
bars[min_idx].set_color('red')
bars[max_idx].set_color('green')

plt.tight_layout()
plt.show()

---

## 練習題（已完成，請閱讀理解）

### 練習 1：視覺化 CNN 學到的 Filter

**目標：** 看看 CNN 第一層學到了什麼樣的 filter

**Hint：** 第一層的 filter 通常學到邊緣、顏色等低階特徵

In [ ]:
# 練習 1：視覺化 filters

# 取得第一層卷積的權重
first_conv = model.conv1
weights = first_conv.weight.data.cpu()

print(f"第一層 filter shape: {weights.shape}")
print(f"  = (out_channels, in_channels, kH, kW)")

# 視覺化前 32 個 filters
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
axes = axes.flatten()

for i in range(32):
    # 取一個 filter，shape = (3, 3, 3)
    filt = weights[i]
    # 正規化到 0-1
    filt = (filt - filt.min()) / (filt.max() - filt.min())
    # 轉成 (H, W, C) 格式
    filt = filt.permute(1, 2, 0)
    
    axes[i].imshow(filt)
    axes[i].axis('off')
    axes[i].set_title(f'Filter {i}')

plt.suptitle('First Conv Layer Filters (learned edge/color detectors)', fontsize=14)
plt.tight_layout()
plt.show()

### 練習 2：視覺化 Feature Maps

**目標：** 看看圖片經過卷積後變成什麼樣子

**Hint：** Feature map 顯示了「哪些位置觸發了這個 filter」

In [ ]:
# 練習 2：視覺化 feature maps

# 取一張測試圖片
img, label = test_dataset[0]
img_input = img.unsqueeze(0).to(device)

# 獲取中間層的輸出
model.eval()

# 第一層 conv 後的 feature maps
with torch.no_grad():
    x = model.conv1(img_input)
    x = model.bn1(x)
    x = F.relu(x)

feature_maps = x.cpu().squeeze()
print(f"Feature maps shape: {feature_maps.shape}")

# 顯示原圖和前 15 個 feature maps
fig, axes = plt.subplots(2, 8, figsize=(16, 4))
axes = axes.flatten()

# 原圖
orig = img.cpu()
orig = orig * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
orig = orig + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
orig = orig.clamp(0, 1)
axes[0].imshow(orig.permute(1, 2, 0))
axes[0].set_title(f'Original\n({classes[label]})')
axes[0].axis('off')

# Feature maps
for i in range(15):
    axes[i+1].imshow(feature_maps[i], cmap='viridis')
    axes[i+1].set_title(f'FM {i}')
    axes[i+1].axis('off')

plt.suptitle('Feature Maps after First Conv Layer', fontsize=14)
plt.tight_layout()
plt.show()

### 練習 3：比較有無資料增強的效果

**目標：** 證明資料增強可以減少過擬合

**Hint：** 無增強時，train acc 會很高但 test acc 較低

In [ ]:
# 練習 3：比較有無資料增強

# 無增強的 transform
no_aug_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# 載入無增強的資料
train_no_aug = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=no_aug_transform
)
train_loader_no_aug = DataLoader(train_no_aug, batch_size=128, shuffle=True)

# 訓練函數（快速版）
def quick_train(model, train_loader, test_loader, epochs=10):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    history = {'train_acc': [], 'test_acc': []}
    
    for epoch in range(epochs):
        # Train
        model.train()
        correct, total = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        history['train_acc'].append(100. * correct / total)
        
        # Test
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        history['test_acc'].append(100. * correct / total)
        
        print(f"Epoch {epoch+1}: Train {history['train_acc'][-1]:.1f}%, Test {history['test_acc'][-1]:.1f}%")
    
    return history

# 訓練兩個模型
print("=" * 50)
print("訓練無資料增強的模型：")
print("=" * 50)
torch.manual_seed(42)
model_no_aug = SmallResNet()
history_no_aug = quick_train(model_no_aug, train_loader_no_aug, test_loader, epochs=10)

print("\n" + "=" * 50)
print("訓練有資料增強的模型：")
print("=" * 50)
torch.manual_seed(42)
model_with_aug = SmallResNet()
history_with_aug = quick_train(model_with_aug, train_loader, test_loader, epochs=10)

In [ ]:
# 比較結果

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 無增強
ax = axes[0]
ax.plot(history_no_aug['train_acc'], 'b-', label='Train')
ax.plot(history_no_aug['test_acc'], 'r-', label='Test')
ax.fill_between(range(10), history_no_aug['train_acc'], history_no_aug['test_acc'], 
                alpha=0.3, color='red', label='Overfitting Gap')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Without Data Augmentation')
ax.legend()
ax.grid(True)

# 有增強
ax = axes[1]
ax.plot(history_with_aug['train_acc'], 'b-', label='Train')
ax.plot(history_with_aug['test_acc'], 'r-', label='Test')
ax.fill_between(range(10), history_with_aug['train_acc'], history_with_aug['test_acc'], 
                alpha=0.3, color='green', label='Smaller Gap')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy (%)')
ax.set_title('With Data Augmentation')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

print(f"\n無增強 - 最終 Train: {history_no_aug['train_acc'][-1]:.1f}%, Test: {history_no_aug['test_acc'][-1]:.1f}%")
print(f"有增強 - 最終 Train: {history_with_aug['train_acc'][-1]:.1f}%, Test: {history_with_aug['test_acc'][-1]:.1f}%")
print(f"\n觀察：無增強時 Train-Test 差距更大（過擬合），有增強時差距較小且 Test 準確率更高")

### 練習 4：使用預訓練模型 (Transfer Learning)

**目標：** 使用 torchvision 提供的預訓練 ResNet

**Hint：** 預訓練模型已經學到了通用的視覺特徵，微調會更快更好

In [ ]:
# 練習 4：Transfer Learning

from torchvision.models import resnet18, ResNet18_Weights

# 載入預訓練的 ResNet18
pretrained_model = resnet18(weights=ResNet18_Weights.DEFAULT)

# 修改最後一層以適應 CIFAR-10（10 類）
# 原本是 1000 類（ImageNet）
pretrained_model.fc = nn.Linear(pretrained_model.fc.in_features, 10)

# 因為 CIFAR-10 是 32x32，需要調整第一層
# 原本 ResNet 假設輸入是 224x224
pretrained_model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
pretrained_model.maxpool = nn.Identity()  # 移除 maxpool（圖片太小）

pretrained_model = pretrained_model.to(device)

print(f"預訓練 ResNet18 參數量: {sum(p.numel() for p in pretrained_model.parameters()):,}")

# 快速訓練
print("\n" + "=" * 50)
print("使用預訓練 ResNet18：")
print("=" * 50)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pretrained_model.parameters(), lr=0.0001)  # 較小的學習率

history_pretrained = {'train_acc': [], 'test_acc': []}

for epoch in range(5):  # 只訓練 5 個 epoch
    pretrained_model.train()
    correct, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = pretrained_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    history_pretrained['train_acc'].append(100. * correct / total)
    
    pretrained_model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = pretrained_model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    history_pretrained['test_acc'].append(100. * correct / total)
    
    print(f"Epoch {epoch+1}: Train {history_pretrained['train_acc'][-1]:.1f}%, "
          f"Test {history_pretrained['test_acc'][-1]:.1f}%")

print(f"\n預訓練模型只用 5 個 epoch 就達到 {history_pretrained['test_acc'][-1]:.1f}% 準確率！")

## Module 3 中場總結

### 核心概念

1. **為什麼用 CNN**：
   - 局部連接 + 權重共享 = 參數少
   - 平移不變性
   - 利用影像的空間結構

2. **卷積運算**：
   - kernel/filter 在圖片上滑動，做內積
   - 重要參數：kernel_size, stride, padding
   - 輸出尺寸公式：`(input - kernel + 2*padding) / stride + 1`

3. **池化層**：
   - 降低維度，增加感受野
   - Max Pooling vs Average Pooling
   - Global Average Pooling 取代全連接

4. **殘差連接（ResNet）**：
   - `output = F(x) + x`
   - 解決深網路訓練困難的問題

5. **資料增強**：
   - 增加資料多樣性，減少過擬合
   - 常用：RandomCrop, HorizontalFlip, ColorJitter

---

## Part 8：進階 CNN 概念

### 8.1 感受野（Receptive Field）

**概念：** 每個輸出神經元能「看到」的輸入區域大小

**重要性：** 
- 感受野太小 → 無法捕捉大範圍的模式
- 感受野太大 → 可能失去局部細節

**計算公式：** 
- 對於連續的 3×3 卷積：RF = 1 + 2 × (層數)
- stride 會快速增加感受野

In [ ]:
# 感受野計算

def calculate_receptive_field(layers):
    """
    計算一系列卷積層後的感受野
    layers: list of (kernel_size, stride, padding)
    """
    rf = 1  # 起始感受野
    stride_product = 1  # 累積 stride
    
    print("感受野計算：")
    print("-" * 50)
    
    for i, (k, s, p) in enumerate(layers):
        rf = rf + (k - 1) * stride_product
        stride_product *= s
        print(f"Layer {i+1}: kernel={k}, stride={s} -> RF = {rf}")
    
    return rf

# 例子：VGG-style 網路
print("VGG-style (多個 3x3 卷積 + maxpool):")
vgg_layers = [
    (3, 1, 1), (3, 1, 1), (2, 2, 0),  # Conv, Conv, Pool
    (3, 1, 1), (3, 1, 1), (2, 2, 0),  # Conv, Conv, Pool
    (3, 1, 1), (3, 1, 1), (2, 2, 0),  # Conv, Conv, Pool
]
rf_vgg = calculate_receptive_field(vgg_layers)
print(f"\n最終感受野: {rf_vgg}×{rf_vgg}")

print("\n" + "=" * 50)
print("\nResNet-style (有 stride=2 的卷積):")
resnet_layers = [
    (7, 2, 3),  # Initial conv
    (3, 2, 1),  # MaxPool
    (3, 1, 1), (3, 1, 1),  # Block 1
    (3, 2, 1), (3, 1, 1),  # Block 2 (downsample)
    (3, 2, 1), (3, 1, 1),  # Block 3 (downsample)
]
rf_resnet = calculate_receptive_field(resnet_layers)
print(f"\n最終感受野: {rf_resnet}×{rf_resnet}")

### 8.2 Grad-CAM：CNN 可解釋性

**問題：** CNN 是「黑盒子」，我們想知道它「看哪裡」做決策

**Grad-CAM：** 利用梯度找出對預測最重要的區域
1. 對目標類別計算梯度
2. 找出哪些 feature map 對該類別最重要
3. 用這些權重組合 feature maps，得到熱力圖

In [ ]:
# 簡化版 Grad-CAM 實現

class GradCAM:
    """簡化版 Grad-CAM"""
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # 註冊 hooks
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)
    
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    
    def generate(self, input_image, target_class=None):
        # Forward pass
        output = self.model(input_image)
        
        if target_class is None:
            target_class = output.argmax(dim=1)
        
        # Backward pass
        self.model.zero_grad()
        one_hot = torch.zeros_like(output)
        one_hot[0, target_class] = 1
        output.backward(gradient=one_hot)
        
        # 計算權重：對梯度做 global average pooling
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        
        # 加權組合 feature maps
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = F.relu(cam)  # 只保留正值
        
        # 正規化到 0-1
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-8)
        
        # Resize 到原圖大小
        cam = F.interpolate(cam, size=input_image.shape[2:], mode='bilinear', align_corners=False)
        
        return cam.squeeze().cpu().numpy()

# 使用 Grad-CAM
model.eval()

# 獲取最後一個卷積層
target_layer = model.layer3[-1].conv2  # SmallResNet 的最後一個卷積

grad_cam = GradCAM(model, target_layer)

# 對幾張測試圖片生成 CAM
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for row in range(3):
    img, label = test_dataset[row * 10]
    img_input = img.unsqueeze(0).to(device)
    img_input.requires_grad = True
    
    # 生成 CAM
    cam = grad_cam.generate(img_input)
    
    # 取得預測
    with torch.no_grad():
        pred = model(img_input.detach()).argmax(dim=1).item()
    
    # 原圖
    orig = img.cpu()
    orig = orig * torch.tensor([0.2023, 0.1994, 0.2010]).view(3, 1, 1)
    orig = orig + torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    orig = orig.clamp(0, 1).permute(1, 2, 0).numpy()
    
    # 顯示
    axes[row, 0].imshow(orig)
    axes[row, 0].set_title(f'Original\nTrue: {classes[label]}')
    axes[row, 0].axis('off')
    
    axes[row, 1].imshow(cam, cmap='jet')
    axes[row, 1].set_title('Grad-CAM')
    axes[row, 1].axis('off')
    
    # 疊加
    axes[row, 2].imshow(orig)
    axes[row, 2].imshow(cam, cmap='jet', alpha=0.5)
    axes[row, 2].set_title(f'Overlay\nPred: {classes[pred]}')
    axes[row, 2].axis('off')
    
    # 只顯示高激活區域
    mask = cam > 0.5
    masked = orig * mask[:, :, np.newaxis]
    axes[row, 3].imshow(masked)
    axes[row, 3].set_title('High Activation Region')
    axes[row, 3].axis('off')

plt.suptitle('Grad-CAM Visualization: Where does the model look?', fontsize=14)
plt.tight_layout()
plt.show()

### 8.3 深度可分離卷積（Depthwise Separable Convolution）

**問題：** 標準卷積參數量太大

**解決方案：** 把卷積分成兩步
1. **Depthwise Conv**：每個通道單獨做卷積
2. **Pointwise Conv**：用 1×1 卷積混合通道

**用途：** MobileNet, EfficientNet 等輕量級網路

In [ ]:
# 深度可分離卷積 vs 標準卷積

class DepthwiseSeparableConv(nn.Module):
    """深度可分離卷積"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        # Depthwise: 每個通道單獨卷積
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size,
                                   stride=stride, padding=padding, 
                                   groups=in_channels)  # groups=in_channels!
        # Pointwise: 1x1 卷積混合通道
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1)
    
    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

# 比較參數量
in_ch, out_ch = 64, 128
kernel_size = 3

# 標準卷積
standard_conv = nn.Conv2d(in_ch, out_ch, kernel_size, padding=1)
standard_params = sum(p.numel() for p in standard_conv.parameters())

# 深度可分離卷積
dw_sep_conv = DepthwiseSeparableConv(in_ch, out_ch, kernel_size)
dw_sep_params = sum(p.numel() for p in dw_sep_conv.parameters())

print("參數量比較：")
print(f"標準卷積 (in={in_ch}, out={out_ch}, k={kernel_size}):")
print(f"  參數量 = {in_ch} × {out_ch} × {kernel_size}² + {out_ch} = {standard_params:,}")
print(f"\n深度可分離卷積:")
print(f"  Depthwise: {in_ch} × 1 × {kernel_size}² + {in_ch} = {in_ch * kernel_size**2 + in_ch}")
print(f"  Pointwise: {in_ch} × {out_ch} × 1 + {out_ch} = {in_ch * out_ch + out_ch}")
print(f"  總參數量 = {dw_sep_params:,}")
print(f"\n深度可分離卷積只需要 {dw_sep_params/standard_params*100:.1f}% 的參數！")

# 測試輸出
x = torch.randn(1, in_ch, 32, 32)
print(f"\n輸入 shape: {x.shape}")
print(f"標準卷積輸出: {standard_conv(x).shape}")
print(f"深度可分離卷積輸出: {dw_sep_conv(x).shape}")

---

## Module 3 完整總結

### 🎯 核心概念

| 概念 | 說明 | 重要程度 |
|------|------|----------|
| **卷積運算** | Filter 滑動做內積，提取局部特徵 | ⭐⭐⭐⭐⭐ |
| **權重共享** | 同一 filter 用於整張圖，大幅減少參數 | ⭐⭐⭐⭐⭐ |
| **池化層** | 降維、增加感受野、平移不變性 | ⭐⭐⭐⭐ |
| **殘差連接** | output = F(x) + x，解決深網路訓練問題 | ⭐⭐⭐⭐⭐ |
| **資料增強** | 增加多樣性，減少過擬合 | ⭐⭐⭐⭐⭐ |
| **BatchNorm** | 穩定訓練，加速收斂 | ⭐⭐⭐⭐ |

### 🔑 關鍵公式

```
卷積輸出尺寸：
output_size = (input - kernel + 2×padding) / stride + 1

感受野計算：
RF_new = RF_old + (kernel - 1) × stride_product

參數量（標準卷積）：
params = in_channels × out_channels × kernel_h × kernel_w + out_channels

參數量（深度可分離卷積）：
params = in_ch × kernel² + in_ch × out_ch   # 約 1/kernel² 的壓縮
```

### 🏗️ 經典架構演進

```
LeNet (1998)    → 首個成功的 CNN
AlexNet (2012)  → ImageNet 突破，使用 ReLU、Dropout、GPU
VGG (2014)      → 堆疊 3×3 卷積，證明深度重要
GoogLeNet (2014)→ Inception module，多尺度特徵
ResNet (2015)   → 殘差連接，深度可達 152+ 層
DenseNet (2017) → 密集連接，特徵重用
MobileNet (2017)→ 深度可分離卷積，輕量化
EfficientNet (2019)→ 複合縮放，效率與準確率平衡
```

### 💡 實務技巧

1. **資料增強必備三件套**：
   - RandomCrop + padding
   - RandomHorizontalFlip
   - Normalize（使用資料集的均值/標準差）

2. **現代 CNN 標準模組**：
   ```
   Conv → BatchNorm → ReLU → Dropout(optional)
   ```

3. **常見超參數**：
   - kernel_size: 3×3（小但有效）
   - stride: 1（保持尺寸）或 2（降維）
   - padding: 'same' 或 (kernel-1)//2

4. **遷移學習優先**：
   - 優先使用預訓練模型
   - 小資料集：凍結前面層，只訓練分類器
   - 大資料集：全部微調，但用較小學習率

5. **調試技巧**：
   - 用 Grad-CAM 檢查模型「看哪裡」
   - 視覺化 filter 和 feature maps
   - 監控每類準確率，找出困難類別

### 🚀 下一步

- Module 4：RNN 與序列資料（文字、時間序列）
- 後續可以探索：Vision Transformer (ViT)、自監督學習 (SimCLR)

---

**恭喜完成 CNN 模組！** 🎉

CNN 是深度學習最重要的基礎架構之一，掌握了卷積、池化、殘差連接這些概念，你就能理解大部分影像相關的模型。